<a href="https://colab.research.google.com/github/francoriboratig/TP2-Datos/blob/master/Features.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

%pylab inline

url='https://raw.githubusercontent.com/JulioCastillo1/TP1-Datos/master/train.csv'
tweets = pd.read_csv(url)
tweets.head()

Populating the interactive namespace from numpy and matplotlib


,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [2]:
tweets['character_count'] = tweets['text'].apply(len)
tweets.head()

,id,keyword,location,text,target,character_count
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,69
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,38
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,133
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,65
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,88


In [ ]:
def word_count(text):
  return len(text.split())
  
tweets['word_count'] = tweets['text'].apply(word_count)
tweets.head()

,id,keyword,location,text,target,character_count,word_count
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,69,13
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,38,7
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,133,22
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,65,8
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,88,16


In [ ]:
def cw_fire(text):
  returnable = 0
  if (text.find('fire') != -1):
    returnable = 1
  return returnable

tweets['contains_word_fire'] = tweets['text'].apply(cw_fire)
tweets.head()

,id,keyword,location,text,target,character_count,word_count,contains_word_fire
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,69,13,0
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,38,7,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,133,22,0
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,65,8,1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,88,16,1


In [ ]:
def cw_storm(text):
  returnable = 0
  if (text.find('storm') != -1):
    returnable = 1
  return returnable

def cw_flood(text):
  returnable = 0
  if (text.find('flood') != -1):
    returnable = 1
  return returnable

def cw_death(text):
  returnable = 0
  if (text.find('death') != -1):
    returnable = 1
  return returnable

def cw_love(text):
  returnable = 0
  if (text.find('love') != -1):
    returnable = 1
  return returnable

tweets['contains_word_storm'] = tweets['text'].apply(cw_storm)
tweets['contains_word_flood'] = tweets['text'].apply(cw_flood)
tweets['contains_word_death'] = tweets['text'].apply(cw_death)
tweets['contains_word_love'] = tweets['text'].apply(cw_love)
tweets.head()

,id,keyword,location,text,target,character_count,word_count,contains_word_fire,contains_word_storm,contains_word_flood,contains_word_death,contains_word_love
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,69,13,0,0,0,0,0
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,38,7,1,0,0,0,0
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,133,22,0,0,0,0,0
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,65,8,1,0,0,0,0
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,88,16,1,0,0,0,0


In [9]:
tweets['has_mentions'] = tweets['text'].str.contains('@')
tweets['amount_of_exclamation_marks'] = tweets['text'].str.count('\!')
tweets['amount_of_interrogation_marks'] = tweets['text'].str.count('\?')
tweets.iloc[0:20]

,id,keyword,location,text,target,character_count,has_mentions,amount_of_exclamation_marks,amount_of_interrogation_marks
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,69,False,0,0
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,38,False,0,0
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,133,False,0,0
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,65,False,0,0
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,88,False,0,0
5,8,NaN,NaN,#RockyFire Update => California Hwy. 20 closed...,1,110,False,0,0
6,10,NaN,NaN,#flood #disaster Heavy rain causes flash flood...,1,95,False,0,0
7,13,NaN,NaN,I'm on top of the hill and I can see a fire in...,1,59,False,0,0
8,14,NaN,NaN,There's an emergency evacuation happening now ...,1,79,False,0,0
9,15,NaN,NaN,I'm afraid that the tornado is coming to our a...,1,52,False,0,0
